In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

	model_C_096, input_features_C_096 = pre_models.Input_C_096('weights/')

	outputs = ['Input_C_096']

	preprocessings = ['skewness']

	models = [model_C_096]

	input_features = [ input_features_C_096]

	result = {}
	for i in range(len(outputs)):

		if preprocessings[i] == 'skewness':
			X = skewness_pure(X_df, input_features[i])
		elif preprocessings[i] == 'min_max_normalization':
			X = min_max_normalization_pure(X_df, input_features[i])

		result[outputs[i]] = predict_mul(models[i], X)
    
	result_df = pd.DataFrame(result, columns = outputs)
	return result_df



In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=12
data=pd.read_csv('./BayesianRidge_Pre_0/Input_C_096.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
nn=data.drop(['Predict'], axis=1)
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_C_094', 'Input_C_099', 'Input_C_098', 'Input_C_044',
       'Input_A2_014', 'Input_C_137', 'Input_C_121', 'Input_C_056',
       'Input_C_101', 'Input_A4_019', 'Input_C_031_Y', 'Input_C_093'],
      dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [56]:
nn_ave(nn, y)

test_rmse_ave: 0.007427233982483053
[0.006681846529997053, 0.011844938531856267, 0.007041727865877138, 0.007187415155773952, 0.006926349531619511, 0.007263675185310165, 0.00881269357530182, 0.006010358468748524, 0.00586764280942508, 0.00663569217092102]


test_r2_ave: -0.23402689328426002
[-0.008371512301302841, -2.0925024544836153, -0.008882987588859192, 0.007472973144797779, -0.01694446664498428, -0.1007669256713637, -0.7023768300937492, 0.1667232999916537, 0.28669462918290367, 0.12868534162191858]


## Linear Regression Baseline

In [57]:
lr_rmse_ave_fea(skew_data,12)

train_rmse: 0.006143586064844172
test_rmse: 0.0063764056586489825
test_r2: 0.1687728929586918


## Lasso

In [58]:
laso_rmse_ave(skew_data[title],y,5e-05)

test_rmse_ave: 0.006292975925823191
[0.006338504676476967, 0.006062411531315351, 0.006756448502576744, 0.006159655198779472, 0.006777498442074015, 0.005943017246401346, 0.006384401133030058, 0.0058201665761850995, 0.006100849185901555, 0.0065868067654912986]


test_r2_ave: 0.16096551814725918
[0.09259492247607826, 0.1899062932838167, 0.07120628928242556, 0.27102995290097476, 0.026295263958347692, 0.26312041870611547, 0.10653263795386536, 0.2186253495261854, 0.22886798208450976, 0.14147607130027273]


## ElasticNet

In [59]:
ElasticNet_rmse_ave(skew_data[title],y,[0.01],0.01)

test_rmse_ave: 0.006289273290400697
[0.006309869575024675, 0.006067455378125738, 0.0067317150049022376, 0.00623161448908959, 0.00675453300122084, 0.005966898073577268, 0.006331983949333754, 0.005779588843976547, 0.0061072339791669854, 0.006611840609589328]


test_r2_ave: 0.16241116069293454
[0.10077506759844945, 0.18855775787606954, 0.0779939586952878, 0.2538982884239481, 0.03288284869335634, 0.2571865138211631, 0.1211434957756572, 0.22948273088664783, 0.2272530938293067, 0.13493785132945957]


## SVR

In [60]:
model=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,12,model)

train_rmse: 0.006294405985045472
test_rmse: 0.006587524105666151
test_r2: 0.1105445248595451


## XGB

In [61]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 80, 
              max_depth = 7, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.6,
              colsample_bytree = 0.2,
              reg_alpha =  0,
              reg_lambda = 0.07)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.006418931640769529
[0.006252188009250737, 0.006621446162239365, 0.0066363919088872105, 0.006649586531495434, 0.006642294653586028, 0.005336185116075906, 0.0061808889490985635, 0.006762445119202418, 0.006279821163890022, 0.006828068793969611]


test_r2_ave: 0.1243925504114165
[0.11714042128428848, 0.033615101299133476, 0.10392084562870063, 0.15045544006979705, 0.06475648878032503, 0.4059208428816128, 0.16258595200099646, -0.05486284523962848, 0.18296114672780794, 0.07743211068113176]


## RandomForest

In [62]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 9,
            min_samples_split = 3,
            max_leaf_nodes = 5,
            min_samples_leaf =6,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.00627877481303907
[0.006270561187322562, 0.0063954843652325455, 0.0063529031646757375, 0.0065320570710004685, 0.006338191133403963, 0.005798670049022241, 0.006055455500053276, 0.006164245252022048, 0.006212951229945163, 0.006667229177712691]


test_r2_ave: 0.1656756548571546
[0.1119439139827555, 0.09844689497170656, 0.1788418734674554, 0.18022094041686376, 0.14843247176227892, 0.29848116380152157, 0.19622962618881012, 0.1235072923042122, 0.20026878775105006, 0.12038358392489201]


In [8]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=5e-05)
elastic_mod=ElasticNet(alpha=[0.01], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 80, 
              max_depth = 7, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.6,
              colsample_bytree = 0.2,
              reg_alpha =  0,
              reg_lambda = 0.07)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 9,
            min_samples_split = 3,
            max_leaf_nodes = 5,
            min_samples_leaf =6,
            random_state = 42)  
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('RandomForest', random_mod),('XGB', xgb_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.006160079158792075
[0.006168037660969958, 0.006010256316085925, 0.006563955183147977, 0.006126051860887441, 0.00656389672902168, 0.005686212550318213, 0.006130923643478388, 0.005913415118646483, 0.005971820692390839, 0.006466221832973842]


test_r2_ave: 0.19622194168112364
[0.1407459090920713, 0.2037848857878397, 0.12337563288721243, 0.27896189351789, 0.08670325724864592, 0.32542736332176136, 0.17607026748519883, 0.19338698886703176, 0.2611408164446348, 0.1726224021589503]


In [9]:

stack_mod = StackingRegressor(regressors=[lasso_mod, random_mod, elastic_mod, xgb_mod], 
                           meta_regressor=vote_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.00626836405684231
[0.006148222667274274, 0.006343514415524706, 0.006680359995351421, 0.0062963078291076455, 0.006644221155881704, 0.005327559932974666, 0.006129424247842146, 0.006438249435326548, 0.006152622478137891, 0.006523158411002094]


test_r2_ave: 0.16557251598209724
[0.14625779695464958, 0.11303947295650008, 0.0920079263806508, 0.23832660718418786, 0.06421390217402267, 0.4078397794454002, 0.17647322327174086, 0.043854279926120454, 0.2157244197138397, 0.15798775181386004]


In [10]:
ave(skew_data[title],y,0.2,stack_mod,0.6,vote_mod,0.2,random_mod)

test_rmse_ave: 0.006117997670088208
[0.006090297125564393, 0.006070846441091593, 0.006457454234426359, 0.006169351941355025, 0.0064511285047046795, 0.0055603960363582935, 0.006029500080220046, 0.005984683134084379, 0.0059367766767179095, 0.006429542526359402]


test_r2_ave: 0.20717097583544844
[0.16226909616352336, 0.18765048439644405, 0.15159152556318944, 0.26873300655944654, 0.1178147024762276, 0.3549490910151162, 0.20310524002801345, 0.17382735572921038, 0.26978696384158074, 0.1819822925817327]


In [12]:
name='Input_C_096'
test_pred=[]
seed=[0,1,3,5,6,7,8,2,9]
stack_w=0.2
vote_w=0.6
model_w=0.2
model=random_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)


final_test = pd.DataFrame(np.mean(test_pred, axis=0), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))
